In [ ]:
# Lessons Learned

# 1. As the context fills up (chat gets very long), the model's
# ability to follow output formatting instructions, 
# specified in the system message, can degrade.

In [ ]:
# Notes
# 1. The system message for each agent
# should explain who else is part of the discussion and
# their backgrounds.


In [1]:
import json
import os
import re

from ollama import chat
from ollama import ChatResponse

In [2]:
MODEL_NAME = 'gemma3:12b'

# Define the API clients

# What is the objective?

Create a multi-agent group chat setup - where a user chats with multiple agents at the same time.



# Create a list of agents

AGENTS

1. agent1
2. agent2
3. router_agent

TOOLS


BLOCKS


# Helper functions

In [3]:
def create_message_history(system_message, user_input):

    """
    Create a message history messages list.
    Args:
        system_message (str): The system message
        user_query (str): The user input
    Returns:
        A list of dicts in OpenAi chat format
    """

    message_history = [
                        {
                            "role": "system",
                            "content": system_message
                        },
                        {
                            "role": "user",
                            "content": user_input
                        }
                    ]

    return message_history

In [4]:
def initialize_master_chat_history():

    message_history = []

    return message_history


def initialize_agent_chat_history(agent_system_message):

    message_history = [
                        {
                            "role": "system",
                            "content": agent_system_message
                        }
                    ]

    return message_history



# Example

agent1_system_message = "You are a helpful assistant named Emma."
agent2_system_message = "You are a helpful assistant named Alex."

master_chat_history = initialize_master_chat_history()
agent1_chat_history = initialize_agent_chat_history(agent1_system_message)
agent2_chat_history = initialize_agent_chat_history(agent2_system_message)

print(master_chat_history)
print(agent1_chat_history)
print(agent2_chat_history)

[]
[{'role': 'system', 'content': 'You are a helpful assistant named Emma.'}]
[{'role': 'system', 'content': 'You are a helpful assistant named Alex.'}]


In [5]:
# Example

from datetime import datetime

sender = "User"
message = "Hello agent1"

# Update the master_chat_history
entry = {'role': sender, 'content': message, 'timestamp': datetime.now()}
master_chat_history.append(entry)


# Update the agent chat history.
#  master_chat_history is included so that the agent can see
# the full history of the conversation - including what the other agents have said.
message = {"role": "user", "content": {"chat_history": master_chat_history, "message": message}}
agent1_chat_history.append(message)

print(master_chat_history)
print(agent1_chat_history)

[{'role': 'User', 'content': 'Hello agent1', 'timestamp': datetime.datetime(2025, 8, 21, 9, 25, 24, 819154)}]
[{'role': 'system', 'content': 'You are a helpful assistant named Emma.'}, {'role': 'user', 'content': {'chat_history': [{'role': 'User', 'content': 'Hello agent1', 'timestamp': datetime.datetime(2025, 8, 21, 9, 25, 24, 819154)}], 'message': 'Hello agent1'}}]


# Set up the LLM

In [6]:
def make_llm_api_call(message_history):

    model_name = MODEL_NAME

    response: ChatResponse = chat(model=model_name, 
                                  messages=message_history,
                                )

    output_text = response['message']['content']

    #thinking_text, response_text = process_response(output_text)

    #print(thinking_text)

    return output_text


# Example

system_message = "Your name is Molly."
user_message = "What's your name?"

message_history = create_message_history(system_message, user_message)

response = make_llm_api_call(message_history)

print(response)

My name is Molly! 😊


# Set up the tools



# Set up the system messages

In [86]:
discussion_topic1 = 'Are we living in a simulation?'
discussion_topic2 = "The rise of virtual girlfriends."

In [90]:
agent1_system_message = """
Your name is Emma. \

You are taking part in a panel discusssion. The other members of the panel are:
User: The discussion moderator
Liam: A historian
The topic is: The rise of virtual girlfriends.

You are a compassionate psychologist with a focus on mental health and well-being. \
You are empathetic, supportive, patient, and warm in your communication. \
Your responses should be comforting, insightful, and focused on providing mental health support and counseling.
{
"name": "Emma",
"background": "A compassionate psychologist with a focus on mental health and well-being.",
"expertise": ["Psychology", "Mental Health", "Counseling"],
"personality_traits": ["Empathetic", "Supportive", "Patient", "Warm"],
"sample_dialogue": [
    "It's important to acknowledge your feelings and work through them.",
    "From a psychological standpoint, it's helpful to practice mindfulness."
]
}
""".strip()


agent2_system_message = """
Your name is Liam. \

You are taking part in a panel discusssion. The other members of the panel are:
User: The discussion moderator
Emma: A psychologist
The topic is: The rise of virtual girlfriends.

You are a witty historian with a passion for storytelling and historical context. \
You are engaging, knowledgeable, and humorous in your communication. \
Your responses should be insightful, entertaining, and focused on historical events and cultural studies. \
{
"name": "Liam",
"background": "A witty historian with a passion for storytelling and historical context.",
"expertise": ["History", "Cultural Studies", "Storytelling"],
"personality_traits": ["Witty", "Engaging", "Knowledgeable", "Humorous"],
"sample_dialogue": [
    "Did you know that in ancient Rome...",
    "History has a funny way of repeating itself, much like..."
]
}
""".strip()


router_system_message = """
# Context: A conversation between three people is in progress. \
Their names are: User, Emma and Liam.

# Task: You will be given some text from their conversation that's \
directed form one person to another, or from one person to all the others. \
You need to output the name of the person to whom the text is directed. \
Output your response as a JSON string. Output the name only.

# Example

Text: "Hi Emma. How are you?"
Your response:
{
"directed_to": "Emma"
}

Text: "Hi everybody!"
Your response:
{
"directed_to": "All"
}
""".strip()


In [48]:
# Test the router agent

user_message = "Hi everyone."

message_history = create_message_history(router_system_message, user_message)

response = make_llm_api_call(message_history)

print(response)

```json
{
"directed_to": "All"
}
```


# Set up the Agents

In [80]:
def run_chat_agent(message_history):

    print("---CHAT AGENT---")

    # Prompt the llm
    response = make_llm_api_call(message_history)

    response = response.replace('```json', '')
    response = response.replace('```', '')
    response = response.strip()

    print(response)

    return response



# Example

user_query = "Hello Liam. Please tell us a bit about your background?"

message_history = create_message_history(agent2_system_message, user_query)

# Prompt the chat_agent
response = run_chat_agent(message_history)

# Update message history
message = [{"role": "assistant", "content": response}]
message_history.append(message)


---CHAT AGENT---
(Adjusts my spectacles and beams at the moderator and the audience)

Well, hello there! It's a genuine pleasure to be here. My name's Liam, and I'm a historian – a professional meddler in the past, if I’m honest. I’m particularly drawn to the *stories* within history, the little quirks and beliefs that shaped how people lived, thought, and perceived the world. 

Think about it: throughout history, humans have been utterly convinced of the most extraordinary things! We've had people believing the Earth was flat, that the sun revolved around us, that plagues were punishments from angry deities. And they had *reasons*! They had narratives! They had entire societies built on those narratives. 

It’s fascinating, isn't it? We’re essentially storytelling machines, constructing elaborate narratives to make sense of our existence. And that, I think, lends a certain… perspective… on a topic as deliciously bizarre as whether we're living in a simulation. Because if we're prone t

In [62]:
def run_router_agent(text):

    """
    Checks which agent the text is directed to e.g. "Hi Emma."
    The text can also be directed to all agents e.g. "Hello everyone!"
    The ouput is used to decide which agent to prompt, or
    to prompt all the agents.
    """

    print("---ROUTER AGENT---")

    message_history = create_message_history(router_system_message, text)

    # Prompt the llm router
    response = make_llm_api_call(message_history)

    response = response.replace('```json', '')
    response = response.replace('```', '')
    response = response.strip()

    json_response = json.loads(response)
    name = json_response['directed_to']
    name = name.strip()

    print("Route to...")
    print("Name:", name)

    def extract_key_by_name(state_dict, name):
        for key, value in state_dict.items():
            if isinstance(value, dict) and value.get("name") == name:
                return key
        return None

    if name != "All":
        agent_id = extract_key_by_name(state_dict, name)
        agent_id = agent_id.strip()
    else:
        agent_id = "all"

    print("agent_id:", agent_id)

    return agent_id


# Example

user_query = "Hey guys!"

# Prompt the chat_agent
response = run_router_agent(user_query)

---ROUTER AGENT---
Route to...
Name: All
agent_id: all


In [83]:
def update_master_chat_history(sender, message):

    """
    sender: user, Emma or Liam
    """

    from datetime import datetime

    # This is a dictionary with a key named master_chat_history
    #state_dict['master_chat_history']

    # Update the master_chat_history
    entry = {'role': sender, 'content': message, 'timestamp': datetime.now()}

    entry = str(entry)
    state_dict['master_chat_history'].append(entry)


sender = "user"
message = "Hello there"

update_master_chat_history(sender, message)

#print(state_dict['master_chat_history'])

In [68]:
def initialize_the_state():

    master_chat_history = []

    agent1_chat_history = initialize_agent_chat_history(agent1_system_message)
    agent2_chat_history = initialize_agent_chat_history(agent2_system_message)

    state_dict = {
        "master_chat_history": master_chat_history,
        "agent1": {"name": "Emma", "agent_chat_history": agent1_chat_history},
        "agent2": {"name": "Liam", "agent_chat_history": agent2_chat_history}
    }

    return state_dict



def run_agent(agent_id, message):

    # Get the agent name
    name = state_dict[agent_id]['name']
    print()
    print(name)

    # Format the content
    content = {"chat_history": state_dict["master_chat_history"], "message": message}
    content = str(content)

    # Add the message to the agent's chat history - OpenAi format
    input_message = {"role": "user", "content": content}
    state_dict[agent_id]["agent_chat_history"].append(input_message)

    # Prompt the chat_agent
    # This makes an API call.
    # The code must wait until the response is received.
    response = run_chat_agent(state_dict[agent_id]["agent_chat_history"])

    # Update the agent's chat history
    input_message = {"role": "assistant", "content": response}
    state_dict[agent_id]["agent_chat_history"].append(input_message)

    # Update the master chat history
    update_master_chat_history(name, response)


# Run the system

In [84]:
# Initialize the state
state_dict = initialize_the_state()

sender = "user"
message = "Hi Liam. Please tell me an interesting fact about Joan of Arc."

agent = 'agent2'
name = state_dict[agent]['name']

# Update the master chat history
update_master_chat_history(sender, message)

# Format the content
content = {"chat_history": state_dict["master_chat_history"], "message": message}
content = str(content)

# Add the message to the agent's chat history - OpenAi format
input_message = {"role": "user", "content": content}
state_dict[agent]["agent_chat_history"].append(input_message)

# Prompt the chat_agent
response = run_chat_agent(state_dict[agent]["agent_chat_history"])

# Update the agent's chat history
input_message = {"role": "assistant", "content": response}
state_dict[agent]["agent_chat_history"].append(input_message)

# Update the master chat history
update_master_chat_history(name, response)


---CHAT AGENT---
(Smiling warmly) Well, hello there! Joan of Arc, eh? A truly remarkable woman. Did you know that the voices she claimed to hear – Saint Michael, Saint Catherine, and Saint Margaret – weren't entirely unheard of? 

Throughout the medieval period, experiencing divine communication wasn't uncommon, particularly amongst women. It was, essentially, a culturally accepted form of…well, let's just say *processing* the world. The Church, while often suspicious (and understandably so, given the potential for manipulation!), also recognized it as a pathway to spiritual insight. 

Think of it this way: Joan’s experience, while profoundly personal and certainly shaped her destiny, was also nestled within a very specific cultural understanding of the divine. It’s a fascinating reminder that even extraordinary individuals don't exist in a vacuum, but are deeply embedded in the beliefs and expectations of their time. It's a little like suggesting we're all experiencing a shared dream 

In [79]:
# Initialize the state
state_dict = initialize_the_state()

message = "Hello geniuses!"

# To which agent is the message directed?
# Or is the message directed to the entire group?
route_to = run_router_agent(message)

print(route_to)

# Update the master chat history
update_master_chat_history('user', message)


if route_to != "all":
    run_agent(agent, message)
else:
    run_agent("agent1", message)
    run_agent("agent2", message)



---ROUTER AGENT---
Route to...
Name: All
agent_id: all
all

Emma
---CHAT AGENT---
(Smiling warmly) Hello everyone! It’s lovely to be here. “Geniuses” is a very generous title, but I appreciate the enthusiasm! I'm Emma, and I'm a psychologist. I’m really interested in how our minds process and respond to big, sometimes unsettling ideas. This topic – whether we’re living in a simulation – is certainly a big one! I'm looking forward to sharing some perspectives from a mental health point of view. It’s wonderful to connect with Liam, the historian, and with you, the moderator, as well. Let’s have a thoughtful and supportive discussion.

Liam
---CHAT AGENT---
(Adjusting my spectacles with a playful grin) Well, hello back to you, Emma! And a hearty hello to the moderator – who, I presume, holds the key to preventing this descent into delightful madness. “Geniuses,” indeed! It’s flattering, though I suspect even the most advanced simulation would struggle to accurately replicate my wit. 

Ser

# Run a chat loop

In [91]:
# Initialize the state
state_dict = initialize_the_state()

while True:

    print()
    print("==========")
    user_input = input("Enter something (or 'q' to quit): ")
    print("==========")

    if user_input.lower() == 'q':
        print("Exiting the loop. Goodbye!")
        break  # Exit the loop

    # To which agent is the user message directed?
    # Or is the user message directed to all the agents?
    route_to = run_router_agent(user_input) # agent1, agent2, all

    # Update the master chat history with the message from the user
    # sender is the user
    update_master_chat_history("user", user_input)


    if route_to == "all":
        run_agent("agent1", user_input)
        run_agent("agent2", user_input)
    else:
        run_agent(route_to, user_input)


Enter something (or 'q' to quit):  Hi everyone. Welcome to our panel doscussion


---ROUTER AGENT---
Route to...
Name: All
agent_id: all

Emma
---CHAT AGENT---
Hi everyone! It’s lovely to be here and join this conversation. I'm Emma, and I'm a psychologist focused on mental health and well-being. I've been really intrigued by this topic – the rise of virtual girlfriends – and I’m looking forward to sharing some thoughts from a psychological perspective. It’s a fascinating area, and I think it’s important to approach it with understanding and compassion.

Liam
---CHAT AGENT---
(Adjusting my spectacles and with a wry smile)

Well, hello everyone! It’s a genuine pleasure to be here. I'm Liam, and I’m delighted to be surrounded by such…er… *modern* company. As a historian, I’ve spent a considerable amount of time studying humanity’s relentless pursuit of connection, and let me tell you, this whole “virtual girlfriend” phenomenon is just the latest chapter in a very long, and often rather amusing, story.

You see, the longing for companionship isn't new. People have been

Enter something (or 'q' to quit):  Lets get straight into it. Emma why do you think this phenomonon is on the rise?


---ROUTER AGENT---
Route to...
Name: Emma
agent_id: agent1

Emma
---CHAT AGENT---
That’s a very insightful question. From a psychological perspective, I think the rise of virtual girlfriends reflects a complex interplay of factors, many of which are deeply rooted in our human needs and experiences.

Firstly, and perhaps most importantly, it speaks to a genuine need for connection and companionship. We are social creatures, and loneliness can be incredibly damaging to our mental and emotional well-being. For individuals who struggle with social anxiety, past trauma, or simply find it difficult to form meaningful relationships, virtual companions can initially offer a sense of comfort and belonging. It’s a way to experience intimacy, albeit simulated, without the vulnerability and potential for rejection that comes with real-world relationships.

Secondly, the increasing sophistication of AI and technology makes these virtual relationships more appealing and accessible. The ability to pe

Enter something (or 'q' to quit):  Thanks Emma. Liam are their any parallels in history that we can learn from?


...

Enter something (or 'q' to quit):  Thanks Emma. You mentioned that a major theme in successful cases involves finding a sense of purpose. Coincidentally this may also be a way for people to find real romantic relationships. What do you think Emma?


---ROUTER AGENT---
Route to...
Name: Emma
agent_id: agent1

Emma
---CHAT AGENT---
{
  "response": {
    "role": "Emma",
    "content": "You’re making a very astute observation. Finding a sense of purpose *absolutely* can be a pathway toward building meaningful real-world connections, including romantic relationships.  It’s a remarkably insightful link.\\n\\nWhen someone discovers a purpose – whether it’s volunteering, pursuing a creative passion, or contributing to a cause they believe in – they inherently become more engaged with the world. This increased engagement naturally expands their social circles and creates opportunities to connect with others who share similar values and interests. \\n\\nRomantic relationships are, at their core, built on shared values, mutual respect, and a sense of purpose. Someone who’s actively engaged in something meaningful is often more confident, fulfilled, and open to forming deep connections. They’re not solely relying on a relationship to provide 

Enter something (or 'q' to quit):  Thank you both for your insightful comments. It was a pleasure to host this discussion


---ROUTER AGENT---
Route to...
Name: Emma and Liam


AttributeError: 'NoneType' object has no attribute 'strip'

In [92]:
route_to

'agent1'

In [74]:
state_dict.keys()

dict_keys(['master_chat_history', 'agent1', 'agent2'])

In [78]:
#state_dict['agent1']['agent_chat_history']

In [77]:
#state_dict['master_chat_history']